#Matplotlib. My first MEP

**A.K.A. How I became a Matplotlib developper.**

-----------------

-----------------

-----------------

-----------------

-----------------


## MEP22
### Federico Ariza ariza.federico@gmail.com
### https://github.com/fariza
-----------------

-----------------

-----------------

-----------------

-----------------




## Traditional Matplotlib imports + a little magic

In [1]:
%matplotlib tk
import matplotlib
matplotlib.use('tkagg')
matplotlib.rcParams['toolbar'] = 'toolmanager'
import matplotlib.pyplot as plt

## Typical messy plot

In [2]:
import numpy as np
x = np.linspace(0, 2*np.pi, 50)
def get_y(phase):
    y = np.sin(x + phase)
    y2 = y + 0.1 * np.random.normal(size=x.shape)
    return y, y2

fig, ax = plt.subplots()
phases = [0, 0.2, 0.5, 0.7, 2., 3., 4.]

for i, phase in enumerate(phases):
    y, y2 = get_y(phase)
    line = ax.plot(x, y,  gid=str(i), picker=5, lw=4)
    ax.plot(x, y2, 'o', gid=str(i), picker=5, markersize=8)

### Simple Tool

In [3]:
from matplotlib.backend_tools import ToolBase

class ExtraSimple(ToolBase):
    default_keymap = 'C'
    def trigger(self, *args, **kwargs):
        self.toolmanager.message_event("You are doing great!!")


In [4]:
t = fig.canvas.manager.toolmanager.add_tool('simple', ExtraSimple)

In [5]:
fig.canvas.manager.toolbar.add_tool('simple', 'navigation')

## Little bit more complex tool

In [6]:
from matplotlib.backend_tools import ToolToggleBase


class VisibilityTool(ToolToggleBase):

    def __init__(self, *args, **kwargs):
        self.group = kwargs.pop('group')
        ToolToggleBase.__init__(self, *args, **kwargs)

    def enable(self, *args, **kwargs):
        self.set_lines_visibility(False)

    def disable(self, *args, **kwargs):
        self.set_lines_visibility(True)

    def set_lines_visibility(self, state):
        gr_lines = []
        for ax in self.figure.get_axes():
            for line in ax.get_lines():
                if line.get_gid() == self.group:
                    line.set_visible(state)
        self.figure.canvas.draw()


In [7]:
try:
    fig.canvas.manager.toolmanager.remove_tool('nav')
    
except KeyError:
    pass

for i in range(len(phases)):
    key = str(i)
    fig.canvas.manager.toolmanager.add_tool(key, VisibilityTool, group=key)
    fig.canvas.manager.toolmanager.update_keymap(key, key)

## Those colors.....

In [8]:
from matplotlib.backend_tools import ToolToggleBase
from matplotlib import colors
from tkColorChooser import askcolor 

class ThatColor(ToolToggleBase):
    def __init__(self, *args, **kwargs):
        self.pid = None
        ToolToggleBase.__init__(self, *args, **kwargs)
        self.converter = colors.ColorConverter()

    def enable(self, *args, **kwargs):
        self.toolmanager.message_event('enable')
        self.pid = self.figure.canvas.mpl_connect('pick_event', self.onpick)


    def disable(self, *args, **kwargs):
        self.figure.canvas.mpl_disconnect(self.pid)
        self.pid = None

    def onpick(self, event):
        thisline = event.artist
        original = thisline.get_color()
        if original is None:
            original = thisline.get_markerfacecolor()
        original = self.converter.to_rgb(original)
        original = colors.rgb2hex(original)
        color = askcolor(original)[1]
        thisline.set_color(color)
        self.figure.canvas.draw()


In [9]:
fig.canvas.manager.toolmanager.add_tool('color', ThatColor)
fig.canvas.manager.toolbar.add_tool('color', 'io')